# DUC Classification

In [68]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [106]:
# import well header data
well_header = pd.read_csv("../data/WellHeader_Datathon.csv")
# well_header['RigReleaseDate'] = pd.to_datetime(well_header.RigReleaseDate)
well_header[['RigReleaseDate','StatusDate']].apply(pd.to_datetime)
# import well production
well_prod = pd.read_csv("../data/WellProduction.csv")
well_prod['ProdPeriod'] = pd.to_datetime(well_prod.ProdPeriod)
# import perf treatments
well_perfs = pd.read_csv("../data/PerfTreatments.csv")
well_perfs['ActivityDate'] = pd.to_datetime(well_perfs.ActivityDate)

### Explore Well Header

In [107]:
well_header.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10437 entries, 0 to 10436
Data columns (total 89 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   EPAssetsId                 10437 non-null  int64  
 1   Province                   10437 non-null  object 
 2   LicenceNumber              10437 non-null  object 
 3   UWI                        10437 non-null  object 
 4   CurrentOperator            10437 non-null  object 
 5   CurrentOperatorParent      10437 non-null  int64  
 6   CurrentOperatorParentName  10437 non-null  object 
 7   Licensee                   10437 non-null  object 
 8   LicenseeParentCompany      10437 non-null  int64  
 9   LicenseeParentCompanyName  10437 non-null  object 
 10  LicenceDate                10437 non-null  object 
 11  CurrentStatus              10437 non-null  object 
 12  CurrentStatusStandardised  10437 non-null  object 
 13  WellType                   10437 non-null  obj

In [109]:
# remove un-necesary columns
well_header_clean = well_header[[
                'EPAssetsId',
                # 'CurrentOperator',
                # 'CurrentStatus',
                # 'Licensee',
                # 'DrillingContractor',
                'Surf_Longitude','Surf_Latitude',
                'WellProfile',
                'CurrentStatus',
                'StatusDate',
                'RigReleaseDate',
                ]]

well_header_clean

,EPAssetsId,Surf_Longitude,Surf_Latitude,WellProfile,CurrentStatus,StatusDate,RigReleaseDate
0,2587301,-117.529866,54.747632,Horizontal,Gas Lift,2/28/2016,12/21/2015
1,2593278,-117.527042,54.737446,Horizontal,Gas Lift,9/23/2016,6/10/2016
2,2579437,-117.559923,54.730864,Horizontal,Gas Lift,11/24/2015,1/22/2015
3,2591761,-117.560141,54.730703,Horizontal,Gas Lift,2/17/2016,1/26/2016
4,2587302,-117.561283,54.723339,Horizontal,Gas Lift,2/21/2016,1/11/2016
...,...,...,...,...,...,...,...
10432,2619592,-111.199446,51.895432,Directional,Pumping,3/13/2019,2/11/2019
10433,2621490,-111.098455,51.959143,Directional,Pumping,9/7/2019,8/15/2019
10434,2614697,-114.367997,51.272852,Directional,Pumping,11/13/2018,9/10/2018
10435,2613170,-114.336463,51.257185,Directional,Pumping,10/29/2018,8/26/2018


In [88]:
well_header_clean.CurrentStatus.value_counts()

Active       5818
Pumping      2943
Flowing      1446
Gas Lift      157
Suspended      73
Name: CurrentStatus, dtype: int64

### Explore Production

In [15]:
well_prod.ProdType.value_counts()

Production Hours                279435
Gas Production (MMcf)           279091
Oil Production (Bbls)           196902
Water Production (Bbls)         166025
Condensate Production (Bbls)     39601
Name: ProdType, dtype: int64

In [16]:
well_prod.shape

(961054, 5)

In [17]:
well_prod.head()

,EPAssetsId,ProdPeriod,ProdType,Volume,WellHeader.Match
0,1159612,2019-12-31,Production Hours,744.0,Yes
1,1166272,2019-12-31,Production Hours,744.0,Yes
2,1172352,2019-12-31,Production Hours,744.0,Yes
3,1152904,2019-12-31,Production Hours,744.0,Yes
4,1160926,2019-12-31,Production Hours,744.0,Yes


In [110]:
well_prod_grouped = well_prod.groupby(['EPAssetsId','ProdType'],as_index=False).agg({'ProdPeriod':['first','max'],'Volume':'sum'})
well_prod_grouped.columns = well_prod_grouped.columns.get_level_values(0)
well_prod_grouped.columns = ['EPAssetsId','ProdType','ProdPeriod_first','ProdPeriod_last','ProdType_sum'] 

Get difference in time column:

In [111]:
well_prod_grouped['ProdPeriod_Total'] = (well_prod_grouped['ProdPeriod_last'] - well_prod_grouped['ProdPeriod_first']).dt.days

In [112]:
well_prod_grouped

,EPAssetsId,ProdType,ProdPeriod_first,ProdPeriod_last,ProdType_sum,ProdPeriod_Total
0,3373,Condensate Production (Bbls),2017-09-30,2019-12-31,48923.322360,822
1,3373,Gas Production (MMcf),2019-03-31,2019-12-31,1627.348362,275
2,3373,Production Hours,2019-01-31,2019-12-31,17450.400000,334
3,3373,Water Production (Bbls),2017-09-30,2019-12-31,24642.178440,822
4,6976,Condensate Production (Bbls),2015-11-30,2019-12-31,4753.830840,1492
...,...,...,...,...,...,...
37721,2625202,Gas Production (MMcf),2020-01-31,2020-01-31,87.216962,0
37722,2625202,Oil Production (Bbls),2020-01-31,2020-01-31,14318.729700,0
37723,2625908,Gas Production (MMcf),2020-01-31,2020-01-31,0.017657,0
37724,2625908,Oil Production (Bbls),2020-01-31,2020-01-31,1430.300520,0


### Explore Well Perfs

In [77]:
well_perfs = well_perfs[['EPAssetsId','ActivityDate','ActivityType']]

In [96]:
well_perfs_grouped = well_perfs.groupby(['EPAssetsId','ActivityType'],as_index=False).agg({'ActivityDate':['first']})
well_perfs_grouped.columns = well_perfs_grouped.columns.get_level_values(0)
well_perfs_grouped.columns = ['EPAssetsId','ActivityType','ActivityDate_first'] 
well_perfs_grouped

,EPAssetsId,ActivityType,ActivityDate_first
0,3373,Fracture,2017-07-29
1,3373,Perforation,2017-07-29
2,6976,Fracture,2015-10-30
3,6976,Perforation,2015-10-30
4,6977,Fracture,2015-10-31
...,...,...,...
15168,2624627,Multi-Stage Fracture,2020-01-30
15169,2624628,Multi-Stage Fracture,2020-01-28
15170,2624670,Multi-Stage Fracture,2020-01-19
15171,2624670,Open Hole/Barefoot Completion,2020-01-10


### Merge all data